# Note:
- This notebook file may contain methods or algorithms that are NOT covered by the teaching content of BT4222 and hence will not be assessed in your midterm exam.
- It serves to increase your exposure in depth and breath to the practical methods in addressing the specific project topic. We believe it will be helpful for your current project and also your future internship endeavors.

# **Import Library**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Define Network Structure**
The network structure was from "Deep-Sentiment: Sentiment Analysis Using Ensemble of CNN and Bi-LSTM Models" Figure1. Also, you can add three or as much as you want models to increase the accuracy. We just take 2 models as an example



In [ ]:
class Net_cnn_lstm(nn.Module):
    def __init__(self):
        super(Net_cnn_lstm, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 3, 1,1, bias=True)
        self.Bn1 = nn.BatchNorm1d(32)
        #########################################

        self.bi_lstm1 = nn.LSTM(input_size=50, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)
        self.bi_lstm2 = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)

        self.conv1 = nn.Conv1d(1, 32, 3, 1,1, bias=True)
        self.Bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.AvgPool1d(kernel_size=4, stride=4)

        self.fc1 = nn.Linear(1024, 100, bias=True)
        self.fc2 = nn.Linear(100, 5, bias=True)



    def forward(self, x):
        x = torch.flatten(x, 1)
        x, _ = self.bi_lstm1(x)
        x, _ = self.bi_lstm2(x)
        x = x.view(-1, 1, 128)
        x = F.relu(self.Bn1(self.conv1(x)))
        x = self.pool1(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

class Net_CNN(nn.Module):
    def __init__(self):
        super(Net_CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 3, 1,1, bias=True)
        self.Bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(32, 32, 3, 1,1, bias=True)
        self.Bn2 = nn.BatchNorm1d(32)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)


        self.fc1 = nn.Linear(32*12, 5, bias=True)
        self.dropout3 = nn.Dropout2d(0.3)


    def forward(self, x):
        x = F.relu(self.Bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.Bn2(self.conv2(x)))
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

class Net_LSTM(nn.Module):
    def __init__(self):
        super(Net_LSTM, self).__init__()

        #########################################

        self.bi_lstm1 = nn.LSTM(input_size=50, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)
        self.bi_lstm2 = nn.LSTM(input_size=128, hidden_size=128, num_layers=1, batch_first=True, bidirectional=False)
        self.fc1 = nn.Linear(128, 5, bias=True)


    def forward(self, x):
        x = torch.flatten(x, 1)
        x, _ = self.bi_lstm1(x)
        x, _ = self.bi_lstm2(x)
        x= self.fc1 (x)
        return x

# **Training and Testing**

In [ ]:
def test(model1,model2, model3,device, test_loader):
    model1.eval()
    model2.eval()
    model3.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target-1
            output = (model1(data)+model2(data)+model3(data))/3
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    return correct

# **Hyperparameter**



In [ ]:
class Args:
  epochs = 0
  lr = 1.0
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  no_cuda = False
  seed = 1

args = Args()

# **Load Data**

In [ ]:
from google.colab import drive

import gdown

file_id = '1CCIfElCaURQbuYvHZiL445UQIRzmmuM7'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'train_vectors.pt'
gdown.download(url, output, quiet=False)

file_id = '1bwkg7XdmH6Mkp_tkAakCbxJMWNAXJU43'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'train_labels.pt'
gdown.download(url, output, quiet=False)

file_id = '1fprUkqC9Qb-y1eDRZt0gA4-4gS941TUo'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'test_vectors.pt'
gdown.download(url, output, quiet=False)

file_id = '1VwOqpW7DZPhqAGDrreVwhtzCB2lUc_LD'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'test_labels.pt'
gdown.download(url, output, quiet=False)


train_vectors = torch.load('train_vectors.pt')
train_labels = torch.load('train_labels.pt')
test_vectors = torch.load('test_vectors.pt')
test_labels = torch.load('test_labels.pt')


file_id = '1q16FOtZw62Q84lfwqKaHPJpT8p7EVHAs'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Combination_lstm_CNN.pt'
gdown.download(url, output, quiet=False)

file_id = '1ulkKik-gvtP5k00U79em7vUPczygXzM2'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Baseline_lstm.pt'
gdown.download(url, output, quiet=False)

file_id = '1iWa39Ya8ggvgt8wIDOqIuEhALxtawOzK'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Baseline_CNN.pt'
gdown.download(url, output, quiet=False)




Downloading...
From: https://drive.google.com/uc?id=1CCIfElCaURQbuYvHZiL445UQIRzmmuM7
To: /content/train_vectors.pt
100%|██████████| 80.0M/80.0M [00:00<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bwkg7XdmH6Mkp_tkAakCbxJMWNAXJU43
To: /content/train_labels.pt
100%|██████████| 3.20M/3.20M [00:00<00:00, 177MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fprUkqC9Qb-y1eDRZt0gA4-4gS941TUo
To: /content/test_vectors.pt
100%|██████████| 20.0M/20.0M [00:00<00:00, 113MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1VwOqpW7DZPhqAGDrreVwhtzCB2lUc_LD
To: /content/test_labels.pt
100%|██████████| 801k/801k [00:00<00:00, 101MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q16FOtZw62Q84lfwqKaHPJpT8p7EVHAs
To: /content/Combination_lstm_CNN.pt
100%|██████████| 1.32M/1.32M [00:00<00:00, 128MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ulkKik-gvtP5k00U79em7vUPczygXzM2
To: /content/Baseline_lstm.pt
100%|██████████| 902k/902k [00:00<00:00, 9

'Baseline_CNN.pt'

# **Start training and testing**

In [ ]:
torch.manual_seed(args.seed)

device = torch.device("cpu")
model_cnn_lstm = Net_cnn_lstm().to(device)
model_cnn_lstm.load_state_dict(torch.load('Combination_lstm_CNN.pt', map_location=device), strict=False)

train_dataset = torch.utils.data.TensorDataset(train_vectors, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_vectors, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=640, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=640, shuffle=False)

model_cnn = Net_CNN().to(device)
model_cnn.load_state_dict(torch.load('Baseline_CNN.pt', map_location=device), strict=False)

model_LSTM = Net_LSTM().to(device)
model_LSTM.load_state_dict(torch.load('Baseline_lstm.pt', map_location=device), strict=False)

test(model_cnn,model_LSTM,model_cnn_lstm, device, test_loader)


Test set: Average loss: 0.0000, Accuracy: 56423/100000 (56%)



56423